# EDA for reservations of every day

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

PROJ_ROOT = Path.cwd().parent

if str(PROJ_ROOT) not in sys.path:
    sys.path.append(str(PROJ_ROOT))

In [3]:
import pandas as pd

In [4]:
from hotels.data_local import DataLoaderLocal
from hotels.processing import enrich_reservation_data

df = enrich_reservation_data(DataLoaderLocal().load_raw_data())

In [5]:
df["arrival_date"].min(), df["arrival_date"].max()

(Timestamp('2015-07-01 00:00:00'), Timestamp('2017-08-31 00:00:00'))

In [6]:
enrich_reservation_data(df)
df.sample(n=6, random_state=9)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,n_nights,departure_date,total_transaction,is_last_minute_cancellation,actual_departure_date,n_stay_actual,is_early_departure,breakfast,lunch,dinner
70359,City Hotel,1,150,2017,June,24,15,1,3,3,...,4,2017-06-19,675.0,False,NaT,NaN,False,True,False,False
7328,Resort Hotel,1,143,2016,August,32,3,2,5,2,...,7,2016-08-10,1617.0,False,NaT,NaN,False,True,False,True
55574,City Hotel,1,174,2016,August,34,16,2,8,2,...,10,2016-08-26,1317.5,False,NaT,NaN,False,True,False,True
97166,City Hotel,0,37,2016,September,37,10,2,1,2,...,3,2016-09-13,256.5,False,2016-09-13,3.0,False,True,False,False
117047,City Hotel,0,242,2017,July,30,26,0,3,2,...,3,2017-07-29,447.3,False,2017-07-29,3.0,False,True,False,False
49939,City Hotel,1,288,2016,April,17,21,0,2,2,...,2,2016-04-23,124.0,False,NaT,NaN,False,True,False,False


In [7]:
df.groupby(["is_canceled", "reservation_status"]).size().rename("count").reset_index()

,is_canceled,reservation_status,count
0,0,Check-Out,75011
1,1,Canceled,42989
2,1,No-Show,1206


reservation_status_date

Date at which the last status was set. This variable can be used in conjunction with the ReservationStatus to understand when was the booking canceled or when did the customer checked-out of the hotel

### Cancellation on the arrival date

If the reservation is cancelled before the date of arrival, it is an ordinary cancellation and it does not appear the hotel book. But the guests which are cancelled on the arrival date (i.e. last minutes cancelllation) are expected to stay at the hotel on the date and the hotel must make the rooms for the guests available. The cases of no-show are marked in the data set (`reservation_status = "No-Show"`), but the last-minute cancelations are not marked. 

In [8]:
(
    df.query("reservation_status == 'Canceled'")
    .assign(days_before_cancel=lambda x: x["arrival_date"] - x["reservation_status_date"])["days_before_cancel"]
    .describe()
)

count                         42989
mean     88 days 07:46:34.733536486
std      90 days 19:04:59.040798300
min                 0 days 00:00:00
25%                19 days 00:00:00
50%                56 days 00:00:00
75%               130 days 00:00:00
max               526 days 00:00:00
Name: days_before_cancel, dtype: object

In [9]:
df["is_last_minute_cancellation"] = (df["reservation_status"] == "Canceled") & (
    df["arrival_date"] == df["reservation_status_date"]
)
df.query("is_last_minute_cancellation").sample(n=6, random_state=3)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,n_nights,departure_date,total_transaction,is_last_minute_cancellation,actual_departure_date,n_stay_actual,is_early_departure,breakfast,lunch,dinner
60788,City Hotel,1,215,2016,November,48,21,1,1,2,...,2,2016-11-23,294.00,True,NaT,NaN,False,True,False,False
9989,Resort Hotel,1,0,2017,February,5,2,0,1,1,...,1,2017-02-03,43.00,True,NaT,NaN,False,True,False,False
66971,City Hotel,1,1,2017,April,17,27,0,1,2,...,1,2017-04-28,166.50,True,NaT,NaN,False,True,False,False
61172,City Hotel,1,0,2016,December,49,3,2,1,2,...,3,2016-12-06,341.01,True,NaT,NaN,False,True,False,False
78364,City Hotel,1,3,2015,October,41,8,0,1,2,...,1,2015-10-09,65.00,True,NaT,NaN,False,True,False,False
63810,City Hotel,1,1,2017,February,8,22,0,2,1,...,2,2017-02-24,160.00,True,NaT,NaN,False,True,False,False


### Leaving before the departure date

Some of the guests leave the hotel before the planned check-out.

In [10]:
(
    df.query("reservation_status == 'Check-Out'")
    .assign(x=lambda x: x["departure_date"] - x["reservation_status_date"])["x"]
    .value_counts()
    .sort_index()
)

x
0 days     74985
1 days         3
2 days         1
3 days         1
4 days         1
5 days         2
7 days         7
8 days         2
9 days         1
10 days        3
14 days        5
Name: count, dtype: int64

In [11]:
from typing import Optional


def actual_departure_date(row) -> Optional[pd.Timestamp]:
    if row["reservation_status"] == "Check-Out":
        return row["reservation_status_date"]
    else:
        return None

In [12]:
df["actual_departure_date"] = df.apply(actual_departure_date, axis=1)

In [13]:
df["n_stay_actual"] = (df["actual_departure_date"] - df["arrival_date"]).apply(lambda x: x.days)

In [14]:
df["n_stay_actual"]

0         0.0
1         0.0
2         1.0
3         1.0
4         2.0
         ... 
119385    7.0
119386    7.0
119387    7.0
119388    7.0
119389    9.0
Name: n_stay_actual, Length: 119206, dtype: float64

In [15]:
df["arrival_date"].isna().mean()

0.0

In [16]:
df[
    [
        "reservation_status",
        "reservation_status_date",
        "arrival_date",
        "departure_date",
        "actual_departure_date",
        "n_stay_actual",
    ]
]  # .query("reservation_status != 'Check-Out'")

,reservation_status,reservation_status_date,arrival_date,departure_date,actual_departure_date,n_stay_actual
0,Check-Out,2015-07-01,2015-07-01,2015-07-01,2015-07-01,0.0
1,Check-Out,2015-07-01,2015-07-01,2015-07-01,2015-07-01,0.0
2,Check-Out,2015-07-02,2015-07-01,2015-07-02,2015-07-02,1.0
3,Check-Out,2015-07-02,2015-07-01,2015-07-02,2015-07-02,1.0
4,Check-Out,2015-07-03,2015-07-01,2015-07-03,2015-07-03,2.0
...,...,...,...,...,...,...
119385,Check-Out,2017-09-06,2017-08-30,2017-09-06,2017-09-06,7.0
119386,Check-Out,2017-09-07,2017-08-31,2017-09-07,2017-09-07,7.0
119387,Check-Out,2017-09-07,2017-08-31,2017-09-07,2017-09-07,7.0
119388,Check-Out,2017-09-07,2017-08-31,2017-09-07,2017-09-07,7.0


In [17]:
df["is_early_departure"] = df["actual_departure_date"] < df["departure_date"]

In [18]:
df.isna().mean()

hotel                             0.000000
is_canceled                       0.000000
lead_time                         0.000000
arrival_date_year                 0.000000
arrival_date_month                0.000000
arrival_date_week_number          0.000000
arrival_date_day_of_month         0.000000
stays_in_weekend_nights           0.000000
stays_in_week_nights              0.000000
adults                            0.000000
children                          0.000000
babies                            0.000000
meal                              0.000000
country                           0.004010
market_segment                    0.000000
distribution_channel              0.000000
is_repeated_guest                 0.000000
previous_cancellations            0.000000
previous_bookings_not_canceled    0.000000
reserved_room_type                0.000000
assigned_room_type                0.000000
booking_changes                   0.000000
deposit_type                      0.000000
agent      

In [19]:
min_date, max_date = df["arrival_date"].min(), df["arrival_date"].max()
dates_range = pd.date_range(start=min_date, end=max_date, freq="D")
print(f"{min_date:%Y-%m-%d}--{max_date:%Y-%m-%d} ({len(dates_range)} days)")

2015-07-01--2017-08-31 (793 days)


## Hotel status at the end of the days

- How many guests stay at the hotel in which type of room? How much did they pay for the room? Age groups? 

In [20]:
date = dates_range[100]
date

Timestamp('2015-10-09 00:00:00')

In [21]:
cols = [
    "reservation_status",
    "reservation_status_date",
    "arrival_date",
    "departure_date",
    "actual_departure_date",
    "n_stay_actual",
]
df_sub = df.query("is_canceled == 0").query("arrival_date <= @date < actual_departure_date")
df_sub[cols]

,reservation_status,reservation_status_date,arrival_date,departure_date,actual_departure_date,n_stay_actual
1655,Check-Out,2015-10-23,2015-09-07,2015-10-23,2015-10-23,46.0
1790,Check-Out,2015-10-12,2015-09-14,2015-10-12,2015-10-12,28.0
2025,Check-Out,2015-10-11,2015-09-27,2015-10-11,2015-10-11,14.0
2044,Check-Out,2015-10-10,2015-09-28,2015-10-10,2015-10-10,12.0
2059,Check-Out,2015-10-11,2015-09-29,2015-10-11,2015-10-11,12.0
...,...,...,...,...,...,...
78533,Check-Out,2015-10-12,2015-10-09,2015-10-12,2015-10-12,3.0
78535,Check-Out,2015-10-12,2015-10-09,2015-10-12,2015-10-12,3.0
78614,Check-Out,2015-10-13,2015-10-09,2015-10-13,2015-10-13,4.0
78723,Check-Out,2015-10-15,2015-10-09,2015-10-15,2015-10-15,6.0


In [22]:
df_sub["adults"].sum(), df_sub["children"].sum(), df_sub["babies"].sum()

(686, 24.0, 5)

In [23]:
df_sub["adr"].sum()

31213.46

In [24]:
import numpy as np

gb_room = df_sub.groupby(["hotel", "assigned_room_type"])

pd.concat(
    [
        gb_room.size().rename("count"),
        gb_room["adults"].sum().rename("adults"),
        gb_room["children"].sum().rename("children").astype(int),
        gb_room["babies"].sum().rename("babies"),
        np.round(gb_room["adr"].mean().rename("adr_per_book"), 2),
        np.round(gb_room["adr"].sum() / gb_room["n_lodgers"].sum(), 2).rename("adr_per_person"),
    ],
    axis=1,
).assign(adults_per_room=lambda x: x["adults"] / x["count"])

count  adults  children  babies  \
hotel        assigned_room_type                                    
City Hotel   A                     125     232         0       1   
             B                      10      17         0       0   
             C                       1       1         0       0   
             D                      54      93         1       0   
             E                       9      15         0       0   
             F                       5      11         4       0   
             G                       1       2         2       0   
Resort Hotel A                      69     127         7       1   
             B                       1       2         0       0   
             C                       6      13         1       1   
             D                      46      87         0       1   
             E                      28      56         0       0   
             F                       7      13         0       0   
             G                       7      14         9       1   
             I                       2       3         0       0   

                                 adr_per_book  adr_per_person  adults_per_room  
hotel        assigned_room_type                                                 
City Hotel   A                          95.41           51.19         1.856000  
             B                          71.80           42.23         1.700000  
             C                          65.00           65.00         1.000000  
             D                         111.49           64.05         1.722222  
             E                         123.89           74.33         1.666667  
             F                         155.81           51.94         2.200000  
             G                         137.44           34.36         2.000000  
Resort Hotel A                          57.93           29.61         1.840580  
             B                          52.50           26.25         2.000000  
             C                          54.77           21.91         2.166667  
             D                          62.17           32.50         1.891304  
             E                          70.92           35.46         2.000000  
             F                          64.43           34.69         1.857143  
             G                         110.67           32.28         2.000000  
             I                           1.20            0.80         1.500000

## Overview of the new guests of the given date

### At the beginn of the day 

- How many bookings? 
- How many adults / childern / babies?
- Which room types are booked? How many?
- For how many people must dinner be supplied?
- Breakfast? Lunch? ← data from the previous date

### At the end of the day 

- check-in rate: How many bookings are not abondoned.
- total revenue
- How many adults / children / babies?
- Which room types are used? How many?
- For how many people are supplied?

In [25]:
import datetime as dt

- reservation_status = "Check-Out" → In the list 
- reservation_status = "Canceled" and is_last_minute_cancellation is True → In the list 
- reservation_status = "Canceled" and is_last_minute_cancellation is False → Not in the list 
- reservation_status = "No-Show" → In the list

In [26]:
df_sub = df.query("arrival_date == @date").query("reservation_status != 'Canceled' or is_last_minute_cancellation")
df_sub

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,n_nights,departure_date,total_transaction,is_last_minute_cancellation,actual_departure_date,n_stay_actual,is_early_departure,breakfast,lunch,dinner
2286,Resort Hotel,0,1,2015,October,41,9,0,1,1,...,1,2015-10-10,58.00,False,2015-10-10,1.0,False,True,False,False
2287,Resort Hotel,0,16,2015,October,41,9,0,1,1,...,1,2015-10-10,48.00,False,2015-10-10,1.0,False,True,False,False
2288,Resort Hotel,0,1,2015,October,41,9,0,1,2,...,1,2015-10-10,67.00,False,2015-10-10,1.0,False,True,False,False
2290,Resort Hotel,0,77,2015,October,41,9,0,2,2,...,2,2015-10-11,112.00,False,2015-10-11,2.0,False,True,False,False
2291,Resort Hotel,0,77,2015,October,41,9,0,2,1,...,2,2015-10-11,84.00,False,2015-10-11,2.0,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78530,City Hotel,0,1,2015,October,41,9,1,2,2,...,3,2015-10-12,417.99,False,2015-10-12,3.0,False,True,False,False
78533,City Hotel,0,1,2015,October,41,9,1,2,2,...,3,2015-10-12,417.99,False,2015-10-12,3.0,False,True,False,False
78535,City Hotel,0,1,2015,October,41,9,1,2,2,...,3,2015-10-12,507.99,False,2015-10-12,3.0,False,True,False,False
78614,City Hotel,0,1,2015,October,41,9,2,2,2,...,4,2015-10-13,580.00,False,2015-10-13,4.0,False,True,False,False


In [27]:
def get_result(row) -> str:
    if row["reservation_status"] == "No-Show":
        return "No-Show"
    elif row["is_last_minute_cancellation"]:
        return "Last-Minute-Cancellation"
    elif row["n_stay_actual"] == 0:
        return "Check-Out"
    else:
        return "Check-In"


# df_sub.apply(get_result, axis=1).value_counts()

In [28]:
df_sub["result"] = df_sub.apply(get_result, axis=1)
df_sub["result"].value_counts()

result
Check-In                    131
Last-Minute-Cancellation      2
Check-Out                     1
Name: count, dtype: int64

In [29]:
meal2breakfast = {"BB": True, "HB": True, "FB": True, "SC": False}
meal2lunch = {"BB": False, "HB": False, "FB": True, "SC": False}
meal2dinner = {"BB": False, "HB": True, "FB": True, "SC": False}

In [30]:
df_sub["dinner"] = df_sub["meal"].map(meal2dinner)

In [31]:
(df_sub["dinner"] * df_sub["n_lodgers"]).sum()

52.0

In [32]:
(df_sub["dinner"] * df_sub["n_lodgers"])[df_sub["result"] != "Check-In"].sum()

2.0

In [33]:
df_sub.groupby("result", as_index=False)[["adults", "children", "babies"]].sum()

,result,adults,children,babies
0,Check-In,240,9.0,2
1,Check-Out,2,0.0,0
2,Last-Minute-Cancellation,3,0.0,0


In [34]:
gb = df_sub.groupby("reserved_room_type")

pd.concat([gb.size().rename("n_booking"), gb[["adults", "children", "babies"]].sum()], axis=1)

,n_booking,adults,children,babies
reserved_room_type,,,,
A,85,158,4.0,1
B,2,4,2.0,0
C,1,1,0.0,0
D,40,71,0.0,0
E,4,7,0.0,0
G,2,4,3.0,1
